In [1]:
# Load Module & Environment Variables
import psycopg2
from dotenv import load_dotenv
import pandas as pd
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
load_dotenv()

True

In [2]:
# Connect to the database
supabaseconnection = psycopg2.connect(database=os.getenv("SUPABASE_DATABASE"),
                                    host=os.getenv("SUPABASE_HOST"),
                                    user=os.getenv("SUPABASE_USER"),
                                    password=os.getenv("SUPABASE_PASSWORD"),
                                    port=os.getenv("SUPABASE_PORT"))

cursor = supabaseconnection.cursor()

In [3]:
cursor.execute("SELECT * FROM public.survey")

column_headers = [desc[0] for desc in cursor.description]
rows = cursor.fetchall()

cursor.close()
supabaseconnection.close()

survey_raw = pd.DataFrame(rows, columns=column_headers)
survey_raw.replace('\n', ' ', regex=True, inplace=True)
#survey_raw.to_csv('./data/survey_raw_db.csv', index=False)

In [4]:
# user_id represents the user who sent the message
# we need to anonymize this column but we need to keep the mapping
# we can use number 1 to 150 as the replacement
# we can store the mapping in a dictionary
user_id_mapping = {}
for i, user_id in enumerate(survey_raw["user_id"].unique()):
    # maximum user id is three digit, when the user id is less than 3 digit, we need to add leading zero
    user_id_mapping[user_id] = f"{i+1:03}"

# Apply the mapping
survey_raw["user_id"] = survey_raw["user_id"].apply(lambda x: user_id_mapping[x])


# Change the value of topic_id based on the value of topic name in the topic.csv
topic_data = pd.DataFrame(
    {
        # transpose the topic.csv
        "id": [
            "4a20c13b-37bf-4f71-80fd-1562ade45678",
            "4a20c13b-37bf-4f71-80fd-1562adead5c5",
            "4a20c13b-37bf-4f71-80fd-1564adead5c5",
            "4a20c13b-37bf-4f71-80fd-1234adead5c5",
            "1b0c0e52-a4a1-4cca-8a8d-adb8368d2bf2",
            "4a20c13b-37bf-4f71-80fd-15678dead5c5",
            "4a20233b-37bf-4f71-80fd-1564adead5c5",
            "4a20c13b-37bf-4f71-80fd-15656dead5c5",
            "4a20c13b-37bf-4f71-80fd-1514adead5c5",
        ],
        "topic_name": [
            "0",
            "1",
            "2",
            "3",
            "4",
            "5",
            "6",
            "7",
            "8",
        ],
    }
)
topic_data = topic_data[["id", "topic_name"]]
topic_data = topic_data.set_index("topic_name")
topic_data = topic_data.to_dict()["id"]
topic_data = {v: k for k, v in topic_data.items()}
survey_raw["topic_id"] = survey_raw["topic_id"].apply(lambda x: topic_data[x])

# Change created_at to yyyy-mm-dd format
survey_raw["created_at"] = pd.to_datetime(survey_raw["created_at"]).dt.strftime(
    "%Y-%m-%d"
)
# trim the date to yyyy-mm-dd format
survey_raw["created_at"] = survey_raw["created_at"].str.slice(0, 10)

# Delete colums id
del survey_raw["id"]

In [5]:
try:
    # Check if the 'data' directory exists, if not, create it
    if not os.path.exists("data"):
        logging.info("Creating 'data' directory...")
        os.makedirs("data")

    # Save the survey data to csv
    logging.info("Saving survey data to 'data/survey_raw.csv'...")
    survey_raw.to_csv("./data/survey_raw.csv", index=False)
    logging.info("Survey data saved successfully.")
except Exception as e:
    logging.error(f"An error occurred: {str(e)}")

INFO:root:Saving survey data to 'data/survey_raw.csv'...
INFO:root:Survey data saved successfully.
